In [1]:
import pandas as pd
import numpy as np
import re
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')
import os
import time
import gc
import json
from mini_batch_helper import extractor, MiniBatchCorpus

Building prefix dict from /home/nctu/Talk2AI_Contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.uf5db4499c0b7f893953f2b98fec37422.cache
Loading model cost 1.488 seconds.
Prefix dict has been built succesfully.


In [2]:
# reference: https://github.com/dennybritz/chatbot-retrieval/blob/8b1be4c2e63631b1180b97ef927dc2c1f7fe9bea/udc_hparams.py
exp_name = 'dual_lstm_8'
with open('models/%s/model_parameters.json' %exp_name) as json_data:
    params = json.load(json_data)
    json_data.close()

In [3]:
# Read in  training data
word2vec_fname = params['word2vec_model_name']
extra_words = ['<pad>']
unknown_word = None

word2id, id2word, word_p, embedding_matrix, corpus, corpus_id = extractor(word2vec_fname, [], 0, extra_words, unknown_word)

In [4]:
record = {}
save_record_dir = 'models/%s/' %exp_name
record['newest_model_dir'] = 'models/' + exp_name +'/newest/'
record['best_model_dir'] = 'models/' + exp_name +'/best/'
record['loss_train'] = []
record['loss_valid'] = []
record['accuracy_valid'] = []
record['best_iter'] = 0
record['sample_correct'] = 0

In [5]:
# Define model
import tensorflow as tf

# Input
context = tf.placeholder(dtype=tf.int32, shape=(None, None), name='context')
context_len = tf.placeholder(dtype=tf.int32, shape=(None,), name='context_len')
response = tf.placeholder(dtype=tf.int32, shape=(None, None), name='response')
response_len = tf.placeholder(dtype=tf.int32, shape=(None,), name='response_len')
target = tf.placeholder(dtype=tf.int32, shape=(None, ), name='target')
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')


#with tf.device('/gpu:0'):
# Embedding
init_embedding_W = tf.constant_initializer(embedding_matrix)
embeddings_W = tf.get_variable('embeddings_W', shape=[embedding_matrix.shape[0], embedding_matrix.shape[1]], initializer=init_embedding_W)
context_embedded = tf.nn.embedding_lookup(embeddings_W, context, name="embed_context")
response_embedded = tf.nn.embedding_lookup(embeddings_W, response, name="embed_response")

if params['n_layers'] == 1:
# shared LSTM encoder
    cell = tf.nn.rnn_cell.LSTMCell(num_units=params['rnn_dim'], forget_bias=2.0, 
                use_peepholes=True, state_is_tuple=True, reuse=tf.get_variable_scope().reuse)
    cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)
    c_outputs, c_states = tf.nn.dynamic_rnn(cell, context_embedded, dtype=tf.float32)
    mask = tf.expand_dims(tf.one_hot(context_len, depth=tf.shape(context)[1]), 1)
    encoding_context = tf.squeeze(tf.matmul(mask, c_outputs), 1)   # c_states.h
    r_outputs, r_states = tf.nn.dynamic_rnn(cell, response_embedded, dtype=tf.float32)
    mask = tf.expand_dims(tf.one_hot(response_len, depth=tf.shape(response)[1]), 1)
    encoding_response =  tf.squeeze(tf.matmul(mask, r_outputs), 1)  # r_states.h
else:
    cells = [tf.nn.rnn_cell.LSTMCell(num_units=params['rnn_dim'], forget_bias=2.0, use_peepholes=True, state_is_tuple=False, reuse=tf.get_variable_scope().reuse) 
                for _ in range(params['n_layers'])]
    dropcells = [tf.contrib.rnn.DropoutWrapper(cell,input_keep_prob=keep_prob) for cell in cells]
    multicell = tf.contrib.rnn.MultiRNNCell(dropcells, state_is_tuple=False)
    multicell = tf.contrib.rnn.DropoutWrapper(multicell, output_keep_prob=keep_prob)
    c_outputs, c_states = tf.nn.dynamic_rnn(multicell, context_embedded, dtype=tf.float32)
    mask = tf.expand_dims(tf.one_hot(context_len, depth=tf.shape(context)[1]), 1)
    encoding_context = tf.squeeze(tf.matmul(mask, c_outputs), 1)   # c_states.h
    r_outputs, r_states = tf.nn.dynamic_rnn(multicell, response_embedded, dtype=tf.float32)
    mask = tf.expand_dims(tf.one_hot(response_len, depth=tf.shape(response)[1]), 1)
    encoding_response =  tf.squeeze(tf.matmul(mask, r_outputs), 1)  # r_states.h

# σ(cMr)
M = tf.get_variable('M', shape=[params['rnn_dim'], params['rnn_dim']], initializer=tf.truncated_normal_initializer(stddev=0.01))

# "Predict" a  response: c * M
generated_response = tf.matmul(encoding_context, M)
generated_response = tf.expand_dims(generated_response, 2)
encoding_response = tf.expand_dims(encoding_response, 2)

# Dot product between generated response and actual response
logits = tf.matmul(generated_response, encoding_response, True)
logits = tf.reshape(logits, [-1])

# Apply sigmoid to convert logits to probabilities (for prediction, not for loss)
probs = tf.sigmoid(logits)
correct_prediction = tf.logical_or( tf.logical_and(tf.equal(target,1), tf.greater_equal(probs,0.5)), tf.logical_and(tf.equal(target,0), tf.less(probs,0.5)))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Calculate the binary cross-entropy loss
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf.to_float(target)))
loss = loss + params['l1_loss'] * tf.reduce_sum(tf.abs(M))

#train_step = tf.train.AdamOptimizer(params['learning_rate']).minimize(loss)
optimizer = tf.train.AdamOptimizer(params['learning_rate'])
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_norm(grad, params['clip']), var) for grad, var in gvs]
train_step = optimizer.apply_gradients(capped_gvs)

In [6]:
# Load in sample and test
sample = pd.read_csv('datas/sample_test_data.txt')
sample_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
sample_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
sample_y = sample.answer.values
assert(np.sum([len(_)!=6 for _ in sample_x2]) == 0)
sample_x1 = [[word for word in jieba.cut(' '.join(s)) if word != ' '] for s in sample_x1]
sample_x2 = [[[word for word in jieba.cut(r) if word != ' '] for r in rs] for rs in sample_x2]


test_datas = pd.read_csv('datas/AIFirstProblem.txt')
test_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.dialogue.values]
test_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.options.values]
assert(np.sum([len(_)!=6 for _ in test_x2]) == 0)
test_x1 = [[word for word in jieba.cut(' '.join(s)) if word != ' '] for s in test_x1]
test_x2 = [[[word for word in jieba.cut(r) if word != ' '] for r in rs] for rs in test_x2]

In [7]:
def word_lst_2_id_lst(lst, pad_to_len=-1):
    pad_word_id = word2id['<pad>']
    pad_len = max(len(lst), 0)
    id_list = [word2id[lst[i]] if i<len(lst) and lst[i] in word2id else pad_word_id for i in range(pad_len)]
    pad_len = pad_to_len - len(id_list)
    if pad_len > 0:
        id_list.extend([pad_word_id] * pad_len)
    return id_list

In [8]:
pad_to_length = -1

In [9]:
sample_id1 = np.array([word_lst_2_id_lst(s, pad_to_length) for s in sample_x1])
sample_id2 = np.array([[word_lst_2_id_lst(r, pad_to_length) for r in rs] for rs in sample_x2])
test_id1 = np.array([word_lst_2_id_lst(s, pad_to_length) for s in test_x1])
test_id2 = np.array([[word_lst_2_id_lst(r, pad_to_length) for r in rs] for rs in test_x2])

In [10]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, record['best_model_dir']+'model.ckpt')

INFO:tensorflow:Restoring parameters from models/dual_lstm_8/best/model.ckpt


In [11]:
score = []
for q, rs in zip(sample_id1, sample_id2):
    for r in rs:
        now_score = sess.run(probs, {
            context: [q],
            response: [r],
            keep_prob: params['keep_prob_valid'],
            context_len:[len(q)-1],
            response_len:[len(r)-1]})[0]
        score.append(now_score)
#             print(now_score, [id2word[idx] for idx in q], [id2word[idx] for idx in r])
score = np.array(score).reshape(-1, 6)
my_ans = np.argmax(score, axis=1)
sample_correct = np.sum(my_ans == sample_y)
print('sample correct %4d' % (sample_correct), flush=True)
record['sample_correct'] = sample_correct.tolist()

sample correct    8


In [12]:
score

array([[ 0.49914375,  0.50045317,  0.50524336,  0.50411385,  0.50393343,
         0.50720251],
       [ 0.50686544,  0.51499879,  0.50074548,  0.49391925,  0.50516891,
         0.50923228],
       [ 0.50903654,  0.50764638,  0.50769573,  0.50530785,  0.50288779,
         0.50101858],
       [ 0.50604457,  0.50621617,  0.49536687,  0.50626075,  0.49702889,
         0.50689912],
       [ 0.50049418,  0.50183284,  0.49242011,  0.50053787,  0.49925581,
         0.50967932],
       [ 0.49790299,  0.51386374,  0.50519663,  0.50586748,  0.51938707,
         0.50406784],
       [ 0.55156147,  0.5075438 ,  0.48584616,  0.55778337,  0.51230252,
         0.51879293],
       [ 0.50732559,  0.49826074,  0.50446159,  0.5012539 ,  0.50704503,
         0.50931817],
       [ 0.50397664,  0.4952471 ,  0.49991098,  0.49277857,  0.48833892,
         0.50313699],
       [ 0.49768269,  0.4997144 ,  0.50146765,  0.48420268,  0.50164115,
         0.50265551],
       [ 0.50436151,  0.49945605,  0.50297558,  0.